# 피벗 테이블과 그룹분석

In [7]:
import pandas as pd
import numpy as np

In [3]:
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시", "연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [5]:
# index, column, value
df1.pivot('도시','연도','인구' )

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


#### 2) 그룹분석


##### 그룹핑해서 묶고 어떤 함수를 쓸것인가

In [8]:
np.random.seed(0)
df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [9]:
df2.groupby(df2.key1).sum()

,data1,data2
key1,,
A,8,80
B,7,70


In [10]:
df2.groupby(df2.key2).mean()['data2']

,data1,data2
key2,,
one,3,30
two,3,30


In [11]:
df2.groupby(df2.key2)['data2'].mean()

key2
one    30
two    30
Name: data2, dtype: int64

In [12]:
df2.groupby([df2.key1,df2.key2]).sum()

data1  data2
key1 key2              
A    one       6     60
     two       2     20
B    one       3     30
     two       4     40

In [13]:
import seaborn as sns
titanic = sns.load_dataset('titanic')

In [16]:
# 여성의 나이 평균
titanic.age.groupby(titanic.sex).mean()

sex
female    27.915709
male      30.726645
Name: age, dtype: float64

In [ ]:
# 1등실에 탄 여성의 나이 평균 
titanic.age.groupby([titanic.pclass,titanic.sex]).mean()


In [17]:
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [18]:
def peak_to_peak_ratio(x):
    return x.max() / x.min()
    
iris.groupby(iris.species).agg(peak_to_peak_ratio)

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


In [20]:
iris.groupby(iris.species).agg(lambda x: x.max()-x.min)

TypeError: Cannot broadcast np.ndarray with operand of type <class 'method'>

### 3) pivot table method

In [21]:
df1.pivot('도시','연도','인구' )

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


In [26]:
# 순서에 주의할 것 #인구를 도시와 연도 순으로 보여줘라
df1.pivot_table('인구','도시','연도')

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


In [28]:
df1.pivot_table('인구','도시','연도',
                    margins=True, margins_name= '합계') #margins 데이터가 있으면 집계를 하겠다.

연도,2005,2010,2015,합계
도시,,,,
부산,3512547.0,3393191.0,3448737.0,3.451492e+06
서울,9762546.0,9631482.0,9904312.0,9.766113e+06
인천,NaN,263203.0,2890451.0,1.576827e+06
합계,6637546.5,4429292.0,5414500.0,5.350809e+06


- tips 데이터 사례

In [29]:
tips = sns.load_dataset('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [32]:
tips['tip_pct']=np.round(tips.tip / tips.total_bill * 100,2)
tips.tail() 

,total_bill,tip,sex,smoker,day,time,size,tip_pct
239,29.03,5.92,Male,No,Sat,Dinner,3,20.39
240,27.18,2.00,Female,Yes,Sat,Dinner,2,7.36
241,22.67,2.00,Male,Yes,Sat,Dinner,2,8.82
242,17.82,1.75,Male,No,Sat,Dinner,2,9.82
243,18.78,3.00,Female,No,Thur,Dinner,2,15.97


In [33]:
# 성별 데이터 갯수
tips.groupby(tips.sex).count()

,total_bill,tip,smoker,day,time,size,tip_pct
sex,,,,,,,
Male,157,157,157,157,157,157,157
Female,87,87,87,87,87,87,87


In [35]:
tips.groupby('sex').count()

,total_bill,tip,smoker,day,time,size,tip_pct
sex,,,,,,,
Male,157,157,157,157,157,157,157
Female,87,87,87,87,87,87,87


In [36]:
# 남녀에 따라 smoker냐 아니냐에 따라 비율이 나옴 
tips.groupby(['sex','smoker']).size()

sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
dtype: int64

In [42]:
# 성별 팁 비율
tips.groupby('sex')[['tip_pct']].mean()

,tip_pct
sex,
Male,15.764713
Female,16.648276


In [43]:
# 흡연유무에 따른 팁 비율 
tips.groupby('smoker')[['tip_pct']].mean()

,tip_pct
smoker,
Yes,16.319140
No,15.932318


In [45]:
# 성별/흡연유무에 따른 팁 비율
tips.groupby(['sex','smoker'])[['tip_pct']].mean()

tip_pct
sex    smoker           
Male   Yes     15.276667
       No      16.066598
Female Yes     18.214545
       No      15.691111

In [46]:
tips.pivot_table('tip_pct','sex','smoker')

smoker,Yes,No
sex,,
Male,15.276667,16.066598
Female,18.214545,15.691111


In [47]:
# 연습문제 4.7.3
tips.groupby('day')[['tip_pct']].mean()

,tip_pct
day,
Thur,16.126452
Fri,16.991579
Sat,15.314598
Sun,16.689605


In [48]:
tips.groupby('time')[['tip_pct']].mean()

,tip_pct
time,
Lunch,16.411765
Dinner,15.951477


In [49]:
tips.groupby(['day','time'])[['tip_pct']].mean()

tip_pct
day  time             
Thur Lunch   16.129016
     Dinner  15.970000
Fri  Lunch   18.875714
     Dinner  15.892500
Sat  Lunch         NaN
     Dinner  15.314598
Sun  Lunch         NaN
     Dinner  16.689605

In [53]:
tips.pivot_table('tip_pct','time','day')

day,Thur,Fri,Sat,Sun
time,,,,
Lunch,16.129016,18.875714,NaN,NaN
Dinner,15.970000,15.892500,15.314598,16.689605


In [54]:
tips.pivot_table('tip_pct',['time','size'],'day')


day               Thur        Fri        Sat        Sun
time   size                                            
Lunch  1     18.170000  22.380000        NaN        NaN
       2     16.401277  18.196000        NaN        NaN
       3     14.460000  18.770000        NaN        NaN
       4     14.552000        NaN        NaN        NaN
       5     12.140000        NaN        NaN        NaN
       6     17.366667        NaN        NaN        NaN
Dinner 1           NaN        NaN  23.180000        NaN
       2     15.970000  16.267273  15.528679  18.086410
       3           NaN        NaN  15.142222  15.266000
       4           NaN  11.770000  13.828462  15.317222
       5           NaN        NaN  10.660000  15.986667
       6           NaN        NaN        NaN  10.380000